In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from src import dbops
from src.img_preprocessing import clean_profile

In [2]:
# Load model
model = keras.models.load_model("model/clf.h5")
 
# Load profile
profile = keras.preprocessing.image.load_img("profile.jpg", target_size=(256, 256))
profile = keras.preprocessing.image.img_to_array(profile)
profile = clean_profile(tf.constant(profile, dtype=tf.float32))
profile = tf.image.rgb_to_grayscale(profile)
profile = tf.expand_dims(profile, 0)

# Make prediction
prediction = model.predict(profile)
cluster = np.argmax(prediction)
probability = np.max(prediction)
 
# Get active riders from predicted cluster
riders = dbops.fetch_riders()
riders = pd.json_normalize(riders)
riders = riders[(riders["cluster"] == cluster) & (riders["active"])]
riders.drop(columns=["active", "img"], inplace=True)
riders["total_points"] = riders.loc[:, "points.classic":"points.climber"].sum(axis=1)
riders.sort_values("total_points", ascending=False, inplace=True)

# Display prediction info
print(f"Predicted cluster: {cluster} ({probability})".center(80, "-"))
print(f"Top active riders in cluster {cluster}:".center(80, "-"))
display(riders.head(10))

-------------------Predicted cluster: 2 (0.2610403895378113)--------------------
------------------------Top active riders in cluster 2:-------------------------


,name,nationality,weight,height,cluster,points.classic,points.gc,points.tt,points.sprint,points.climber,total_points
259,andre-greipel,Germany,75.0,184.0,2,4529,1883,504,15646,342,22904
137,mark-cavendish,Great Britain,70.0,175.0,2,2432,1326,678,13573,43,18052
466,elia-viviani,Italy,67.0,178.0,2,4144,866,160,8350,320,13840
376,nacer-bouhanni,France,65.0,175.0,2,3916,449,51,8854,376,13646
202,jose-joaquin-rojas,Spain,70.0,177.0,2,2805,1869,130,5897,1546,12247
348,bryan-coquard,France,59.0,171.0,2,1995,900,289,5532,179,8895
401,dylan-groenewegen,Netherlands,70.0,177.0,2,2319,249,13,6290,10,8881
135,heinrich-haussler,Australia,NaN,181.0,2,2758,805,276,4256,298,8393
777,sacha-modolo,Italy,67.0,180.0,2,1691,503,19,5948,190,8351
380,sam-bennett,Ireland,73.0,178.0,2,1764,271,41,5670,246,7992
